<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [42]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import re
from function_library import *
from function_library2 import *

In [43]:
re_behalf = re.compile(r'(.*)[Oo]n [Bb]ehalf')
re_bracket = re.compile(r'(.*)[\[\<\(]mailto\:(.*)[\]\>\)]')
re_bracket2 = re.compile(r'(.*)[\[\<\(](.*)[\]\>\)]')

re_name1 = re.compile(r'.*?([A-Za-z]+\s?[A-Z]?\.?)\s([A-Za-z]+)')
re_name2 = re.compile(r'.*?([A-Za-z]+),\s([A-Za-z]+\s?[A-Z]?\.?)')
re_email = re.compile(r'.*?([a-zA-Z_]*\.?\w+@[a-zA-Z_0-9]*\.?[a-zA-Z_]*\.?[a-zA-Z]{2,3})')


In [46]:
# either the sender or the recipient
# df = pd.read_csv('new_clean_output.csv',index_col = 0)
df = pd.read_csv('output_0211.csv')

# df = df.drop_duplicates(keep='first',subset=['Sent'])
# df = df.reset_index(drop=True)

from_list = df['From'].values.tolist()
to_list = df['To'].values.tolist()
cc_list = df['CC'].values.tolist()

In [51]:
people_list = []
named_email_list = []
name_list = []
# find unique complete person in From section

for f in from_list: 
    if pd.isnull(f):
        continue
    f = f.lower()
    if re_behalf.match(f):
        f = re_behalf.findall(f)[0]
    if re_bracket.match(f):
        tname, temail = re_bracket.findall(f)[0]
        tname=tname.lower().strip()
        temail=temail.lower()
        if re_email.match(temail) and re_email.match(tname) == None:
            email = re_email.findall(temail)[0]
            if len(tname.split()) != 2:
                first_name = tname
                last_name = ' '
            else:
                if re_name1.match(tname):
                    name = re_name1.findall(tname)[0]
                    first_name = name[0]
                    last_name = name[1]
                elif re_name2.match(tname): 
                    name = re_name2.findall(tname)[0]
                    first_name = name[1]
                    last_name = name[0]
                else:
                    first_name = 'unrognized name'
                    last_name = tname
                    print('error: unrognized name:',tname)
                    
            if email not in named_email_list:
                person = (first_name, last_name, email)
                people_list.append(person)
                named_email_list.append(email)
                name_list.append(first_name + ' ' + last_name)
           

error: unrognized name: jeﬀ barbacci
error: unrognized name: prospere•, poldine
error: unrognized name: jeﬀ barbacci
error: unrognized name: prospere•, poldine
error: unrognized name: jeﬀ barbacci


In [52]:
len(people_list)

892

In [53]:
# find unique complete person in TO section
for ts in to_list: 
    if pd.isnull(ts):
        continue
    ts = ts.lower()
    ts = ts.split(';')
    for t in ts:
        t = t.strip("'")
        if re_bracket.match(t) or re_bracket2.match(t):
            if re_bracket.match(t):
                tname, temail = re_bracket.findall(t)[0]
            else:
                tname, temail = re_bracket2.findall(t)[0]
            tname=tname.lower().strip()
            temail=temail.lower()
            if tname == '':
                continue

            if re_email.match(temail) and re_email.match(tname) == None:
                email = re_email.findall(temail)[0]
                if len(tname.split()) != 2:
                    first_name = tname
                    last_name = ' '
                else:
                    if re_name1.match(tname):
                        name = re_name1.findall(tname)[0]
                        first_name = name[0]
                        last_name = name[1]
                    elif re_name2.match(tname): 
                        name = re_name2.findall(tname)[0]
                        first_name = name[1]
                        last_name = name[0]
                    else:
                        first_name = 'unrognized name'
                        last_name = tname
                        print('error: unrognized name:',tname)
                        
                if email not in named_email_list:
                    person = (first_name, last_name, email)
                    people_list.append(person)
                    named_email_list.append(email)
                    name_list.append(first_name + ' ' + last_name)
                   

error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: procrete- greg
error: unrognized name: sco_, tricia
error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: laké laosebikan-buggs
error: unrognized name: arthur"buddy" jacobs
error: unrognized name: charles"charlie" dudley
error: unrognized name: frederick "fred"springer
error: unrognized name: laké laosebikan-buggs


In [54]:
len(people_list)

2821

In [55]:
# find unique complete person in CC section
for ccs in cc_list: 
    if pd.isnull(ccs):
        continue
    ccs = ccs.lower()
    ccs = ccs.split(';')
    for cc in ccs:
        cc = cc.strip("'")
        if re_bracket.match(cc) or re_bracket2.match(cc):
            if re_bracket.match(cc):
                tname, temail = re_bracket.findall(cc)[0]
            else:
                tname, temail = re_bracket2.findall(cc)[0]
            tname=tname.lower().strip()
            temail=temail.lower()
            if tname == '':
                continue

            if re_email.match(temail) and re_email.match(tname) == None:
                email = re_email.findall(temail)[0]
                if len(tname.split()) != 2:
                    first_name = tname
                    last_name = ' '
                else:
                    if re_name1.match(tname):
                        name = re_name1.findall(tname)[0]
                        first_name = name[0]
                        last_name = name[1]
                    elif re_name2.match(tname): 
                        name = re_name2.findall(tname)[0]
                        first_name = name[1]
                        last_name = name[0]
                    else:
                        first_name = 'unrognized name'
                        last_name = tname
                        print('error: unrognized name:',tname)
                        
                if email not in named_email_list:
                    person = (first_name, last_name, email)
                    people_list.append(person)
                    named_email_list.append(email)
                    name_list.append(first_name + ' ' + last_name)
             

In [56]:
len(people_list)

3103

In [57]:
new_from_list = []

unknown_idx = 0
# replace the From section with unique people information

for f in from_list: 
    if pd.isnull(f):
        person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), 'f'+str(unknown_idx)+'_'+'l'+str(unknown_idx))
        unknown_idx = unknown_idx + 1
        new_from_list.append(person)
        continue
    email_exist_flag = 0
    f = f.lower().strip("'")
    email = ''
    first_name = ''
    last_name = ''
    if re_behalf.match(f):
        f = re_behalf.findall(f)[0]
        
    if re_email.match(f):
        email = re_email.findall(f)[0]
        email = email.lower()
        email_exist_flag = 1
    if len(f.split()) != 2:
        first_name = f
        last_name = ' '
    else:
        if re_name1.match(f):
            name = re_name1.findall(f)[0]
            first_name = name[0]
            last_name = name[1]
        elif re_name2.match(f): 
            name = re_name2.findall(f)[0]
            first_name = name[1]
            last_name = name[0]
        elif email_exist_flag == 1:
            pass
        else:
            print('error: cannot find name and email in f')
            print('f:', f)
            first_name = 'fake'
            last_name = f
            
    name = first_name + ' ' + last_name

    
    if email in named_email_list:
        idx = named_email_list.index(email)
        new_from_list.append(people_list[idx])
    elif name in name_list:
        idx = name_list.index(name)
        new_from_list.append(people_list[idx])
    elif email_exist_flag == 1:
        person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
        new_from_list.append(person)
        unknown_idx = unknown_idx + 1
    else:
        person = (first_name, last_name, first_name + '_' + last_name)
        new_from_list.append(person)

error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f:  re: trolley
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: sma_, brian
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in

In [58]:
len(new_from_list)

78869

In [59]:
len(from_list)

78869

In [60]:
new_to_lists = []

# replace the To section with unique people information

for ts in to_list: 
    if pd.isnull(ts):
        new_to_lists.append([])
        continue
    ts = ts.lower()
    ts = ts.split(';')
    new_to_list = []
    for t in ts:
        t = t.strip("'")
        email_exist_flag = 0
        email = ''
        first_name = ''
        last_name = ''
        if re_behalf.match(t):
            t = re_behalf.findall(t)[0]

        if re_email.match(t):
            email = re_email.findall(t)[0]
            email = email.lower()
            email_exist_flag = 1
        if len(t.split()) != 2:
            first_name = t
            last_name = ' '
        else:
            if re_name1.match(t):
                name = re_name1.findall(t)[0]
                first_name = name[0]
                last_name = name[1]
            elif re_name2.match(t): 
                name = re_name2.findall(t)[0]
                first_name = name[1]
                last_name = name[0]
            elif email_exist_flag == 1:
                pass
            else:
                first_name = 'fake'
                last_name = t
                print('error: cannot find name and email in t, make fake name')
                print('t:', t)
#                 break

        name = first_name + ' ' + last_name


        if email in named_email_list:
            idx = named_email_list.index(email)
            new_to_list.append(people_list[idx])
        elif name in name_list:
            idx = name_list.index(name)
            new_to_list.append(people_list[idx])
        elif email_exist_flag == 1:
            person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
            new_to_list.append(person)
            unknown_idx = unknown_idx + 1
        else:
            person = (first_name, last_name, first_name + '_' + last_name)
            new_to_list.append(person)
    new_to_lists.append(new_to_list)

error: cannot find name and email in t, make fake name
t:  davidworrell (work)
error: cannot find name and email in t, make fake name
t:  briandavis (tpd)
error: cannot find name and email in t, make fake name
t:  101 managers
error: cannot find name and email in t, make fake name
t:  katrina@ uwbb.org
error: cannot find name and email in t, make fake name
t:  tom(roni) jennings
error: cannot find name and email in t, make fake name
t:  tashdelmar .
error: cannot find name and email in t, make fake name
t:  narayan.persaud@ comcast.net
error: cannot find name and email in t, make fake name
t:  ramon.alexander@ myfloridahouse.gov
error: cannot find name and email in t, make fake name
t:  montford.bill.web @flsenate.gov
error: cannot find name and email in t, make fake name
t:  davidworrell (work)
error: cannot find name and email in t, make fake name
t:  (e-mail) (e-mail)
error: cannot find name and email in t, make fake name
t:  may@ 101tally.com
error: cannot find name and email in t,

error: cannot find name and email in t, make fake name
t:  davidworrell (work)
error: cannot find name and email in t, make fake name
t:  andersonjr., fred
error: cannot find name and email in t, make fake name
t:  briandavis (tpd)
error: cannot find name and email in t, make fake name
t:  davidworrell (work)
error: cannot find name and email in t, make fake name
t:  tashdelmar .
error: cannot find name and email in t, make fake name
t:  katrina@ uwbb.org
error: cannot find name and email in t, make fake name
t:  ralph (businessfax)
error: cannot find name and email in t, make fake name
t:  fapsc1 .
error: cannot find name and email in t, make fake name
t: 101 restaurant
error: cannot find name and email in t, make fake name
t:  lovetoreadtutoring .
error: cannot find name and email in t, make fake name
t:  tashdelmar .
error: cannot find name and email in t, make fake name
t:  briandavis (tpd)
error: cannot find name and email in t, make fake name
t: sara 'saxner
error: cannot find na

In [61]:
len(to_list)

78869

In [62]:
len(new_to_lists)

78869

In [63]:
new_to_lists[:10]

[[('mark', 'beaudoin', 'mark.beaudoin@talgov.com')],
 [('f5682', 'l5682', 'narayan.persaud@comcast.net')],
 [('f5683', 'l5683', 'kavallone@johnryanco.com'),
  ('f5684', 'l5684', 'kblair1046@aol.com'),
  ('f5685', 'l5685', 'kdavis@kelvinsheatandair.com'),
  ('f5686', 'l5686', 'kday@pettushvac.com'),
  ('kent', 'deeb', 'kdeeb@triadincorporated.net'),
  ('f5687', 'l5687', 'keith@campbellplbg.com'),
  ('f5688', 'l5688', 'keithmcneil@comcast.net'),
  ('f5689', 'l5689', 'ken@metalroofpro.com'),
  ('f5690', 'l5690', 'kenleydesign@gmail.com'),
  (" 'kenneth", ' ', " 'kenneth_ "),
  ('f5691', 'l5691', 'kentc@sanderssecurity.com'),
  ('f5692', 'l5692', 'kentdevelopmentiic@gmail.com'),
  ('f5693', 'l5693', 'kevin593a@yahoo.com'),
  ('f5694', 'l5694', 'kevinbelanger@bellsouth.net'),
  ('f5695', 'l5695', 'key4829@yahoo.com'),
  ('f5696', 'l5696', 'kfaver@favergray.com'),
  ('f5697', 'l5697', 'kgerving@sbcglobal.net'),
  ('f5698', 'l5698', 'kgreen@essexco.com'),
  ('f5699', 'l5699', 'khcontractingll

In [64]:
new_cc_lists = []
# replace the cc section with unique people information

for ccs in cc_list: 
    if pd.isnull(ccs):
        new_cc_lists.append([])
        continue
    ccs = ccs.lower()
    ccs = ccs.split(';')
    new_cc_list = []
    if len(ccs) == 1 and ccs[0].strip() == '':
        new_cc_lists.append(new_cc_list)
        continue
    for cc in ccs:
        cc = cc.strip("'")
        email_exist_flag = 0
        email = ''
        first_name = ''
        last_name = ''
        if re_behalf.match(cc):
            cc = re_behalf.findall(cc)[0]

        if re_email.match(cc):
            email = re_email.findall(cc)[0]
            email = email.lower()
            email_exist_flag = 1
        if len(cc.split()) != 2:
            first_name = cc
            last_name = ' '
        else:
            if re_name1.match(cc):
                name = re_name1.findall(cc)[0]
                first_name = name[0]
                last_name = name[1]
            elif re_name2.match(cc): 
                name = re_name2.findall(cc)[0]
                first_name = name[1]
                last_name = name[0]
            elif email_exist_flag == 1:
                pass
            else:
                first_name = 'fake'
                last_name = cc
                print('error: cannot find name and email in cc, make fake name')
                print('cc:', cc)

        name = first_name + ' ' + last_name

        if email in named_email_list:
            idx = named_email_list.index(email)
            new_cc_list.append(people_list[idx])
        elif name in name_list:
            idx = name_list.index(name)
            new_cc_list.append(people_list[idx])
        elif email_exist_flag == 1:
            person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
            new_cc_list.append(person)
            unknown_idx = unknown_idx + 1
        else:
            person = (first_name, last_name, first_name + '_' + last_name)
            new_cc_list.append(person)
    new_cc_lists.append(new_cc_list)

error: cannot find name and email in cc, make fake name
cc:  anneliese (or)
error: cannot find name and email in cc, make fake name
cc:  andersonjr., fred
error: cannot find name and email in cc, make fake name
cc:  davidworrell (work)
error: cannot find name and email in cc, make fake name
cc:  anneliese (or)
error: cannot find name and email in cc, make fake name
cc:  anneliese (or)
error: cannot find name and email in cc, make fake name
cc:  briandavis (tpd)
error: cannot find name and email in cc, make fake name
cc:  anneliese (or)
error: cannot find name and email in cc, make fake name
cc:  maisbel(mae) mendez
error: cannot find name and email in cc, make fake name
cc:  briandavis (tpd)
error: cannot find name and email in cc, make fake name
cc:  claudettecromartie (prsnl)
error: cannot find name and email in cc, make fake name
cc:  jose@ 101tally.com
error: cannot find name and email in cc, make fake name
cc:  scott (aventura)
error: cannot find name and email in cc, make fake na

In [65]:
len(cc_list)

78869

In [66]:
len(new_cc_lists)

78869

In [67]:
new_cc_lists[:10]

[[('kim', 'rivers', 'kim@inkbridge.com')],
 [('skip', 'foster', 'sfoster@tallahassee.com')],
 [('karen', 'jumonville', 'karen.jumonville@talgov.com'),
  ('dale', 'baker', 'dale_baker')],
 [('andrew', 'gillum', 'andrew.gillum@talgov.com')],
 [('[]', ' ', '[]_ ')],
 [('[]', ' ', '[]_ ')],
 [('f74149', 'l74149', 'richel225@gmail.com')],
 [('allen', 'secreast', 'allen.secreast@talgov.com'),
  ('wayne', 'tedder', 'wayne.tedder@talgov.com'),
  ('greg', 'wilkerson', 'greg_wilkerson'),
  ('eric', 'gooch', 'eric_gooch')],
 [('jon', 'brown', 'brownjon@leoncountyfl.gov'),
  ('mathieu', 'cavell', 'cavellm@leoncountyfl.gov'),
  ('victoria', 'connell', 'victoria_connell'),
  ('deborah', 'craig', 'deborah.craig@talgov.com'),
  ('lakendra', 'cunningham', 'lakendra_cunningham'),
  ('stephanie', 'holloway', 'holloways@leoncountyfl.gov'),
  ('dontae', 'iverson', 'dontae_iverson'),
  ('jeremy', 'davis', 'jeremyd@moorecommgroup.com'),
  ('jean', 'mccarter', 'jean_mccarter'),
  ('ken', 'morris', 'morrisk@le

In [68]:
df_new = df.copy()
df_new['From'] = new_from_list
df_new['To'] = new_to_lists
df_new['CC'] = new_cc_lists
df_new


,From,Sent,To,CC,Bcc,Subject,Attachments,Importance,isThread,isAutoMessage,isDisplacement,hasAllCapLine,hasBadDate,Body
0,"(davisson dunlap iii, , davissoniii@dunlapshi...","Wednesday, February 11, 2015 10:07 AM","[(mark, beaudoin, mark.beaudoin@talgov.com)]","[(kim, rivers, kim@inkbridge.com)]",[],RE: Existing Sidewalk Cafe Lease,[],[],False,False,False,False,False,"['Body:', 'Thank you, Mark.', 'Im attaching th..."
1,"(ruth, draper, ruth_draper)","Thursday, September 24, 2015 at 10:13:40 AM Ea...","[(f5682, l5682, narayan.persaud@comcast.net)]","[(skip, foster, sfoster@tallahassee.com)]",[],thanks for your thoughts - FAMU needs more ac_on,[],[],False,False,False,False,False,"['Body:', 'proctorb@leoncountyFL.gov, andrew.g..."
2,"(doug, moore, doug_moore)","Monday, July 09, 2012 2:18 PM","[(f5683, l5683, kavallone@johnryanco.com), (f5...","[(karen, jumonville, karen.jumonville@talgov.c...",[],"City Of Tallahassee, Building Inspection Staff...",[],[],False,False,False,False,False,"['Body:', 'Sent On behalf of Karen Jumonville,..."
3,"(paul, lamaster, paullamaster250@comcast.net)","July 26, 2015 8:45:10 AM EDT","[(f5820, l5820, lefers@tallahassee.com)]","[(andrew, gillum, andrew.gillum@talgov.com)]",[],Unimpressed with Chicago,[],[],False,False,False,False,False,"['Body:', 'I was totally unimpressed with this..."
4,"(martin, merzer, marty@martinmerzer.com)","Thursday, April 11, 2013 11:41 AM","[(allie, fleming, allison.fleming@talgov.com)]","[([], , []_ )]",[],Downtown Improvement Authority Recommendation ...,[],[],False,False,False,False,False,"['Body:', '', 'isDisplacement: True']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78864,"(jessica, brown, jessicae.brown@talgov.com)","Friday, January 13, 2017 6:39 PM","[(ashley, edwards, ashley.edwards@talgov.com)]","[(daniel, fendley, daniel.fendley@talgov.com)]",[],Re: Edison Outdoor Lawn Photo,[],[],False,False,False,False,False,"['Body:', 'You are SOOO good! Thank you!!', 'S..."
78865,"(f5681, l5681, davissoniii@dunlapship)","November 23, 2015 at 8:03:37 AM HST","[(kim, rivers, kim@inkbridge.com)]","[([], , []_ )]",[],Fwd:,[],[],False,False,False,False,False,"['Body:', 'Doubletree - PorteCochere Easement ..."
78866,"(allie, fleming, allison.fleming@talgov.com)","Tuesday, April 02, 2013 3:36 PM","[(danny, capps, danny_capps), (alison, faris, ...","[([], , []_ )]",[],RE: FYI ... Subscriber option for SM newslette...,[],[],False,False,False,False,False,"['Body:', 'From what I can tell it is still do..."
78867,"(edward, kring, edward.kring@talgov.com)","Wednesday, February 22, 2017 at 3:59:54 PM Eas...","[(paige, carter, paige.tallahasseedowntown@gma...","[([], , []_ )]",[],Trolley mee-ng,[],[],False,False,False,False,False,"['Body:', '', 'isDisplacement: True']"


In [69]:
df_new.to_csv('output_0211_name_standardized.csv', index=0)